### Saved Model - Format

In [ ]:
#!pip install pyyaml h5py

In [8]:
import os
import sys
import yaml
import tensorflow as tf
from tensorflow import keras

In [9]:
from model.dataset import Dataset
from model.fp.nnfp import get_fingerprinter
from model.fp.NTxent_loss_single_gpu import NTxentLoss

In [10]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [15]:
config:str = "default" 
cfg = load_config(config)

cli: Configuration from ./config/default.yaml


In [16]:
if cfg['LOSS']['LOSS_MODE'].upper() == 'NTXENT': # Default
    loss_obj_train = NTxentLoss(
        n_org=cfg['BSZ']['TR_N_ANCHOR'],
        n_rep=cfg['BSZ']['TR_BATCH_SZ'] - cfg['BSZ']['TR_N_ANCHOR'],
        tau=cfg['LOSS']['TAU'])

2024-04-23 10:25:33.992595: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 10:25:33.992771: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 10:25:34.032349: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [17]:
checkpoint_dir = '/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/'
#os.listdir(checkpoint_dir)

In [7]:
latest = tf.train.latest_checkpoint('/home/rodrigo/Documents/neural-audio-fp/logs/checkpoint/CHECKPOINT_BSZ_120/')

In [8]:
model = get_fingerprinter(cfg, trainable=True)

In [47]:
dataset = Dataset(cfg)
train_ds = dataset.get_train_ds(cfg['DATA_SEL']['REDUCE_ITEMS_P'])

In [51]:
model.compile(optimizer='ADAM', loss=loss_obj_train, metrics=['accuracy', 'loss_weights='])

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [10]:
model.load_weights(latest)

AssertionError: Nothing to load. No dependencies have been added to <model.fp.nnfp.FingerPrinter object at 0x7bc3505fcf10> yet.